In [1]:
import pickle

# libraries importing
import numpy as np
import pandas as pd
from tqdm import tqdm
import csv
import matplotlib.pyplot as plt
%matplotlib inline

#from sklearn.preprocessing import MinMaxScaler

from tensorflow import keras
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, RepeatVector, Lambda
from keras.callbacks import EarlyStopping
from keras import backend as K
from tensorflow.keras import losses
import tensorflow as tf
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

import warnings
warnings.filterwarnings("ignore")

c:\Users\gioel\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\gioel\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\gioel\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


SEQUENCE

In [2]:
WINDOW_SIZE=40
def create_sequences(values, time_steps=WINDOW_SIZE):
    output = []
    for i in range(len(values) - time_steps + 1):
        output.append(values[i : (i + time_steps)])
    return np.stack(output)

In [3]:

latent_dim=64

def create_lstm_vae():
    
    x = Input(shape=(WINDOW_SIZE, 19))

    # encoding
    h = LSTM(128,return_sequences=True)(x)
    h = LSTM(64)(h)

    # VAE Z layer
    z_mean = Dense(latent_dim, name='mean')(h)
    z_log_sigma = Dense(latent_dim, name='var')(h)
    
    def sampling(args):
        z_mean, z_log_sigma = args
        epsilon = K.random_normal(shape=(1, latent_dim),
                                  mean=0., stddev=1.)
        return z_mean + z_log_sigma * epsilon

    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_sigma])

    #decoding
    decoder= RepeatVector(WINDOW_SIZE)(z)
    decoder= LSTM(64, return_sequences=True)(decoder)
    decoder= LSTM(128, return_sequences=True)(decoder)
    
    x_decoded_mean= Dense(19)(decoder)
    vae = Model(x, x_decoded_mean)

    # encoder, from inputs to latent space
    encoder = Model(x, [z_mean, z_log_sigma, z], name='encoder')
    #encoder.summary()

    #decoder, from latent space to output
    decoder_input = Input(shape=(latent_dim,))
    _h_decoded = RepeatVector(WINDOW_SIZE)(decoder_input)
    _h_decoded = LSTM(64, return_sequences=True)(_h_decoded)
    _h_decoded= LSTM(128, return_sequences=True)(_h_decoded)
    x_decoded= Dense(19)(_h_decoded)

    dencoder = Model(decoder_input, x_decoded, name='dencoder')
    #dencoder.summary()

    
    def vae_loss(x, x_decoded_mean):
        mse = losses.MeanSquaredError()
        xent_loss = mse(x, x_decoded_mean)
        kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma))
        loss = xent_loss + kl_loss

        return loss

    vae.compile(optimizer='rmsprop', loss=vae_loss)

    return vae,encoder

In [4]:
def arch(data,val):

    BATCH_SIZE = 32
    
    model,encoder= create_lstm_vae()
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=30)
    
    history = model.fit(
        data,
        data,
        epochs=1000,
        batch_size=BATCH_SIZE,
        verbose=1,
        validation_data=(val,val),
        callbacks=[callback]
    )

    return history, model, encoder

APP_TOT

In [6]:


with open(f'./DATA_SPLITTED/app_tot.pkl', 'rb') as f:
    DATA = pickle.load(f)

train=DATA['X_train']
val=DATA['X_val']


history,model,encoder=arch(train,val)

model.save_weights(f"MODEL_VAE/model_app_Win_{WINDOW_SIZE}_early.h5")
encoder.save(f"MODEL_VAE/encoder_app_Win_{WINDOW_SIZE}_early.h5")
with open(f'MODEL_VAE/history_app_Win_{WINDOW_SIZE}_early', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)


Train on 62666 samples, validate on 13423 samples
Epoch 1/1000
62666/62666 [==============================] - 173s 3ms/sample - loss: 0.8006 - val_loss: 0.7819
Epoch 2/1000
62666/62666 [==============================] - 165s 3ms/sample - loss: 0.7791 - val_loss: 0.7848
Epoch 3/1000
62666/62666 [==============================] - 165s 3ms/sample - loss: 0.7708 - val_loss: 0.7639
Epoch 4/1000
62666/62666 [==============================] - 165s 3ms/sample - loss: 0.7627 - val_loss: 0.7598
Epoch 5/1000
62666/62666 [==============================] - 165s 3ms/sample - loss: 0.7556 - val_loss: 0.7408
Epoch 6/1000
62666/62666 [==============================] - 163s 3ms/sample - loss: 0.7496 - val_loss: 0.7489
Epoch 7/1000
62666/62666 [==============================] - 163s 3ms/sample - loss: 0.7443 - val_loss: 0.7390
Epoch 8/1000
62666/62666 [==============================] - 161s 3ms/sample - loss: 0.7374 - val_loss: 0.7361
Epoch 9/1000
62666/62666 [==============================] - 161s 3ms/s

isolation forest

In [7]:
#CALCULATE THRESHOLD
from sklearn.ensemble import IsolationForest

with open(f'./DATA_SPLITTED/app_tot.pkl', 'rb') as f:
    DATA = pickle.load(f)
    
model,encoder=create_lstm_vae()


model.load_weights(filepath=f'MODEL_VAE/model_app_Win_{WINDOW_SIZE}_.h5')
encoder.load_weights(filepath=f'MODEL_VAE/encoder_app_Win_{WINDOW_SIZE}_.h5')


z_mean, z_log_sigma, z_train= encoder.predict(DATA['X_test'])


clf = IsolationForest(n_jobs=-1,contamination=0.15)

clf.fit(z_train)

IsolationForest(contamination=0.15, n_jobs=-1)

In [9]:
TP=0
TN=0
FN=0
FP=0


model,encoder= create_lstm_vae()
model.load_weights(filepath=f'MODEL_VAE/model_app_Win_{WINDOW_SIZE}_.h5')
encoder.load_weights(filepath=f'MODEL_VAE/encoder_app_Win_{WINDOW_SIZE}_.h5')

with open(f'./DATA_SPLITTED/app_tot.pkl', 'rb') as f:
    DATA = pickle.load(f)
train=DATA['X_test']

for i in tqdm(range(2,11)):
    if(i!=7 and i!=8 ):


        TEST=np.load(f'./OUTPUTS_ROOT/data/processed/spark_0_15s/spark_0_trace-scl_std/test{i}.npy',allow_pickle=True)

        ANOMALY=np.load(f'./OUTPUTS_ROOT/data/processed/spark_0_15s/spark_0_trace-scl_std/y_test{i}.npy',allow_pickle=True)

        with open(f'./OUTPUTS_ROOT/data/interim/spark_0_15s/test_info{i}.pkl', 'rb') as f:
            TEST_info= pickle.load(f)
        
        for x in range(len(TEST_info)):


                X_test=create_sequences(TEST[x])

                _,_,z_test=encoder.predict(X_test)

                pred=clf.predict(z_test)
                outlier=pred
                for k in range(len(pred)):
                    outlier[k]=pred[k]*(-1)
                    if(outlier[k]<0):
                        outlier[k]=0
                error=clf.score_samples(z_test)


                le=len(outlier)


                true_= ANOMALY[x][20:le+20]>=1
                
                prediction_ = outlier[:].astype(int)==1
                
                TP = TP+(true_ & prediction_).sum()   
                TN = TN+(~true_ & ~prediction_).sum()  
                FP = FP+(~true_ & prediction_).sum()    
                FN = FN+(true_ & ~prediction_).sum()    

PREC=TP / (TP + FP)
REC = TP/ (TP+FN)
f1=2 * PREC * REC/(PREC + REC)

print(f'F1:{f1}')
print(f'PREC:{PREC}')
print(f'REC:{REC}')

print(f'TP:{TP}')
print(f'TN:{TN}')
print(f'FP:{FP}')
print(f'FN:{FN}')

100%|██████████| 9/9 [00:31<00:00,  3.49s/it]

F1:0.6280923810306567
PREC:0.6186852331606217
REC:0.6377900183608747
TP:3821
TN:40083
FP:2355
FN:2170


KNN

In [10]:
#CALCULATE THRESHOLD
from pyod.models.knn import KNN

with open(f'./DATA_SPLITTED/app_tot.pkl', 'rb') as f:
    DATA = pickle.load(f)
    
model,encoder=create_lstm_vae()


model.load_weights(filepath=f'MODEL_VAE/model_app_Win_{WINDOW_SIZE}_.h5')
encoder.load_weights(filepath=f'MODEL_VAE/encoder_app_Win_{WINDOW_SIZE}_.h5')


z_mean, z_log_sigma, z_train= encoder.predict(DATA['X_test'])


clf_n = KNN(n_neighbors=400, method='mean', metric='euclidean', contamination=0.15)

clf_n.fit(z_train)


KNN(algorithm='auto', contamination=0.15, leaf_size=30, method='mean',
  metric='euclidean', metric_params=None, n_jobs=1, n_neighbors=400, p=2,
  radius=1.0)

In [14]:
TP = 0
TN = 0
FP = 0  
FN = 0

model,encoder= create_lstm_vae()
model.load_weights(filepath=f'MODEL_VAE/model_app_Win_{WINDOW_SIZE}_.h5')
encoder.load_weights(filepath=f'MODEL_VAE/encoder_app_Win_{WINDOW_SIZE}_.h5')

with open(f'./DATA_SPLITTED/app_tot.pkl', 'rb') as f:
    DATA = pickle.load(f)
train=DATA['X_test']

for i in tqdm(range(2,11)):
    if(i!=7):


        TEST=np.load(f'./OUTPUTS_ROOT/data/processed/spark_0_15s/spark_0_trace-scl_std/test{i}.npy',allow_pickle=True)

        ANOMALY=np.load(f'./OUTPUTS_ROOT/data/processed/spark_0_15s/spark_0_trace-scl_std/y_test{i}.npy',allow_pickle=True)

        with open(f'./OUTPUTS_ROOT/data/interim/spark_0_15s/test_info{i}.pkl', 'rb') as f:
            TEST_info= pickle.load(f)
        
        for x in range(len(TEST_info)):


                X_test=create_sequences(TEST[x])

                _,_,z_test=encoder.predict(X_test)

                outlier=clf_n.predict(z_test)
                error=clf_n.predict_proba(z_test)
                

                le=len(outlier)


                true_= ANOMALY[x][20:le+20]>=1
                
                prediction_ = outlier[:].astype(int)==1
                
                TP = TP+(true_ & prediction_).sum()   
                TN = TN+(~true_ & ~prediction_).sum()  
                FP = FP+(~true_ & prediction_).sum()    
                FN = FN+(true_ & ~prediction_).sum()    

PREC=TP / (TP + FP)
REC = TP/ (TP+FN)
f1=2 * PREC * REC/(PREC + REC)

print(f'F1:{f1}')
print(f'PREC:{PREC}')
print(f'REC:{REC}')

print(f'TP:{TP}')
print(f'TN:{TN}')
print(f'FP:{FP}')
print(f'FN:{FN}')
                
       

100%|██████████| 9/9 [03:58<00:00, 26.51s/it]

F1:0.6392681210415201
PREC:0.599366587490103
REC:0.6848612786489746
TP:4542
TN:45044
FP:3036
FN:2090
